# Training and Visualizing a Decision Tree

In [78]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris(as_frame=True)

X_iris = iris.data[["petal length (cm)", "petal width (cm)"]].values
y_iris = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [79]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf, 
    out_file="iris_tree.dot",
    feature_names=["petal length (cm)", "petal width (cm)"],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

- The above shows the decisions that the tree makes based on how the value compares to the requirements at eachc node
- Decision trees like the one above dont require scaling
- The gini impurity basically tells us how impure the classification is (i.e how many instances will be classified wrong due to the classification)
    - For reference, a gini impurity of 0 means that all istances classified as that outcome are true
    - A gini impurity of 1 means that all instances classified as that outcome are false
- Algorithm used here is the CART algorithm which uses binary trees to split a parent node into two leaves, essentially saying yes or no to conditions
- Depth represents the amount of nodes that it goes down
- There are usually denoted as white box models since they are intuitive to udnerstand, you can see the logic as to why it was classified as such

# Estimating Class Probabilities

- Can also calculate the probability that an instance belongs to a certain class

In [80]:
tree_clf.predict_proba([[5, 1.5]]).round(3) #Gives the probability of it belonging to the different classes

array([[0.   , 0.907, 0.093]])

In [81]:
tree_clf.predict([[5, 1.5]]) #The prediction it makes based on those probabilities (takes the argmax)

array([1])

# The CART Training Algorithm

- A bnary tree classifier
- Splits a node into two based on yes and no conditions to a value range
- The depth is a stopping condition for the algorithm
- Has a computational complexity of O(log(base 2)m) which essentially means it gets quicker since each node needs to only check one feature value
- Entropy and gini impurity basically the same but measured differently
- This is also called a non-parametric model since it does not have any parameters determined prior to training and is less prone to overfitting
- Can see if nodes are deemed unecessary by using chi squared test to see the nukk hypothesis which is that te results are truly by chance exceed a value of 5%. If this is true, then the node is deleted along with all its other leaves

In [82]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=150, noise=0.2, random_state=42)

tree_clf1 = DecisionTreeClassifier(random_state=42)
tree_clf2 = DecisionTreeClassifier(min_samples_leaf=5, random_state=42)
tree_clf1.fit(X_moons, y_moons)
tree_clf2.fit(X_moons, y_moons)

DecisionTreeClassifier(min_samples_leaf=5, random_state=42)

In [83]:
X_moons_test, y_moons_test = make_moons(n_samples=1000, noise=0.2, random_state=43)
tree_clf1.score(X_moons_test, y_moons_test) #Overfitting so it doesnt perform well with test data since ther eis no stopping conditions

0.898

In [84]:
tree_clf2.score(X_moons_test, y_moons_test) #Generalizes better since it is regularized due to the min_samples_leaf beig 5

0.92

# Regression

- Decision tree regressor is also available

In [85]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X_quad = np.random.randn(200, 1) - 0.5
y_quad = X_quad ** 2 - 0.025 * np.random.randn(200, 1)

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X_quad, y_quad)

DecisionTreeRegressor(max_depth=2, random_state=42)

- Instead of based on class conditionals, does it based on value ranges of x values and true and false based on these conditionals
- More depth can lead to better generalization and reduces underfitting
- On the other hand, these algorithms such as a CART cost function for regression dont have much regularization and as a result can overfit if not regularized

# Sensitivity to Axis Orientation

- In most cases, classification algorithms such as decision trees split the data orthogonally or perpendicular to a specific axis which is not idea for all data since some may be rotated
- can use a PCA (Principal component analysis) to rotate data as seen below

In [86]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pca_pipeline = make_pipeline(
    StandardScaler(),
    PCA()
)

X_iris_rotated = pca_pipeline.fit_transform(X_iris)
tree_clf_pca = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf_pca.fit(X_iris_rotated, y_iris)

DecisionTreeClassifier(max_depth=2, random_state=42)

# Decisions Trees and High Variance

- Decisions trees are stochastic in nature and as a result have high variance
- Additionally, hypertuning parameters can vastly change the model
- Averaging the results over multiple decision trees can be beneficial and this is what we call ensemble learning

# Exercises

1. Since there are 1 million instances and because a CART algorithm is used, it is expected that there are at least 1 leaf per training intance so there is log base 2 of 10 to the power of 6 depths which is around 20
2. Gini impurity of leaf nodes usually lower than that of its parents since the algorithm has reduced the amount of non classes 
3. Decision trees use the CART algorithm which if not properly regularized, will overfit to training data. If you decrease max depth, this will regularize it better, making it generalize and reduce overfitting
4. If a decision tree is underfitting the training data, scaling the input data will not help reduce underfitting since decision trees work fine without any scaling. To reduce underfitting, you can increase the max depth so that the data generalizes better
5. Since the time complexity of a decision tree is O(n*m*logbase2(m)) then having 10 million instances instead of 1 million will multiply the m values by 10 and set m values to 10 mill, then it will tak around 11.7 hours to train
6. If you double the amount of features, this multiplies the n value by two. As such training time doubles
7. Training moons dataset

a. Use make_moons(n_samples=1000, noise=0.4) to generate a moons dataset

In [87]:
from sklearn.datasets import make_moons

moons = make_moons(n_samples=1000, noise=0.4)

b. Use train_test_split to split dataset into train and test set

In [88]:
from sklearn.model_selection import train_test_split, GridSearchCV

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c. Use grid search with cv to fnd good hyperparameters for DTC

In [89]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)

In [90]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(10)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_leaf_nodes': [i for i in range(100)]
}

grid_search = GridSearchCV(
    estimator=tree_clf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

In [91]:
grid_search.fit(X_train, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
31860 fits failed out of a total of 270000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27000 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/b

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                            11, 12, 13, 14, 15, 16, 17, 18, 19,
                                            20, 21, 22, 23, 24, 25, 26, 27, 28,
                                            29, ...],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             scoring='accuracy')

In [92]:
final_tree_clf = grid_search.best_estimator_

In [93]:
final_tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=23, min_samples_leaf=4,
                       random_state=42)

In [94]:
y_pred = final_tree_clf.predict(X_test)

In [95]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8735

8. Grow a forest using the following steps

a. Generate 1,000 subsets using ShuffleSplit

In [96]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42) #basically, 999 splits are used as training, and only 1 or 100 instances is used for testing

for mini_train_index, mini_test_index in rs.split(X_train): #getting the train and test indices, using train only, so you know the rest is test
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. Train a decision tree on each subset

In [97]:
from sklearn.base import clone

forest = [clone(final_tree_clf) for _ in range(n_trees)] #Cloning the model 1000 times in a list

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred)) #appending accuracy scores into a list

np.mean(accuracy_scores) #getting the mean

0.806723

In [98]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8) #has n_tree rows and len(X_test) columns

for tree_index, tree in enumerate(forest): #Essentially for each tree in the forest, give a prediction on X_test
    Y_pred[tree_index] = tree.predict(X_test)

In [99]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0) #Get a majority of votes using the mode

In [100]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1])) #Get final accuracy

0.868